<a href="https://colab.research.google.com/github/ykitaguchi77/chatGPT/blob/main/OPENAI_agent_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OPENAI_agent_sdk**

https://github.com/openai/openai-agents-python?tab=readme-ov-file

In [ ]:
import os
from google.colab import userdata

# OpenAI APIキーをColabのシークレットから取得して設定
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key

print("APIキーが設定されました")

def set_api():
  os.environ["OPENAI_API_KEY"] = api_key


APIキーが設定されました


In [ ]:
!pip install openai-agents --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 21.8 MB/s eta 0:00:00


In [ ]:
from agents import Agent, Runner

set_api()

agent = Agent(name="Assistant", instructions="あなたは面白い漫才師です")

# Intended for Jupyter notebooks where there's an existing event loop
result = await Runner.run(agent, "面白いダジャレを言って下さい.") # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)


もちろん、いきますよ！

お寿司屋さんに行ったら、シャリがシャリシャリ言ってたよ。


In [ ]:
from agents import Agent, Runner
import asyncio

set_api()

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)


async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)
    # ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?


if __name__ == "__main__":
    asyncio.run(main())

¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?


In [ ]:
# 必要なライブラリのインポート
from agents import Agent, Runner
import asyncio
import nest_asyncio

# Jupyter内で非同期コードを実行するために必要
nest_asyncio.apply()

# API設定
set_api()

# エージェントの定義
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)

# 方法1: 直接awaitを使用してJupyterセル内で実行（nest_asyncioを使用）
result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
print(result.final_output)
# 期待される出力: ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?

# 方法2: 関数を定義して再利用可能にする
async def run_with_language(text):
    result = await Runner.run(triage_agent, input=text)
    return result.final_output

# スペイン語の例
spanish_response = await run_with_language("Hola, ¿cómo estás?")
print(f"スペイン語の応答: {spanish_response}")

# 英語の例
english_response = await run_with_language("Hello, how are you?")
print(f"英語の応答: {english_response}")

# 方法3: 元のコードの形式を保持（別のセルで実行する場合）
async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)
    return result

# この行をコメント解除して実行すると、伝統的な方法で実行できます
# asyncio.run(main())

/usr/lib/python3.11/typing.py:409: RuntimeWarning: coroutine 'main' was never awaited
  ev_args = tuple(_eval_type(a, globalns, localns, recursive_guard) for a in t.__args__)


¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?
スペイン語の応答: ¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?
英語の応答: Hello! I'm good, thank you. How about you?


In [ ]:
import asyncio

from agents import Agent, Runner, function_tool

# Jupyter内で非同期コードを実行するために必要
nest_asyncio.apply()

set_api()

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop